In [10]:
!pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [11]:
from typing_extensions import TypedDict
from typing import Optional

class State(TypedDict):
    graph_state: str
    user_name: str
    mood: Optional[str]
    previous_graph_state: Optional[str]

In [12]:
import difflib
from datetime import datetime

def _make_diff(prev: str, new: str) -> str:
    prev_lines = (prev or "").splitlines()
    new_lines = (new or "").splitlines()
    return "\n".join(difflib.unified_diff(prev_lines, new_lines, lineterm=""))

def node_1(state):
    user = state.get("user_name", "User")
    prev = state.get("graph_state", "")
    print(f"---Node 1 (for {user})---")
    new = prev + " I am"
    diff = _make_diff(prev, new)
    return {"graph_state": new, "diff": diff, "last_updated": datetime.utcnow().isoformat(), "mood": "neutral", "previous_graph_state": prev}

def node_2(state):
    user = state.get("user_name", "User")
    prev = state.get("graph_state", "")
    print(f"---Node 2 (happy for {user})---")
    new = prev + " happy!"
    diff = _make_diff(prev, new)
    return {"graph_state": new, "diff": diff, "last_updated": datetime.utcnow().isoformat(), "mood": "happy", "previous_graph_state": prev}

def node_3(state):
    user = state.get("user_name", "User")
    prev = state.get("graph_state", "")
    print(f"---Node 3 (sad for {user})---")
    new = prev + " sad!"
    diff = _make_diff(prev, new)
    return {"graph_state": new, "diff": diff, "last_updated": datetime.utcnow().isoformat(), "mood": "sad", "previous_graph_state": prev}

In [13]:
import random
from typing import Literal

def decide_mood(state) -> Literal["node_2", "node_3"]:
    # Use the state's text to bias the mood decision
    # get the graph_state as a lowercase string (default to empty)
    user_input = str(state.get("graph_state", "")).lower()
    # check for keywords indicating a negative mood
    if any(word in user_input for word in ["sad", "unhappy", "down", "not good"]):
        return "node_3"
    # Otherwise bias slightly toward happy to keep things positive
    return "node_2" if random.random() < 0.6 else "node_3"

In [ ]:
def _merge_state(state, update):
    s = dict(state) if state else {}
    s.update(update or {})
    return s

class GraphRunner:
    def invoke(self, initial_state, max_steps=3):
        nodes = {'node_1': node_1, 'node_2': node_2, 'node_3': node_3}
        state = dict(initial_state) if initial_state else {}
        print('Starting graph run for', state.get('user_name', 'User'))
        outputs = []
        out = node_1(state)
        outputs.append(out)
        state = _merge_state(state, out)
        for i in range(max_steps - 1):
            choice = decide_mood(state)
            print(f'Chosen next node: {choice}')
            node_fn = nodes.get(choice)
            if node_fn is None:
                print('Unknown node', choice)
                break
            out = node_fn(state)
            outputs.append(out)
            state = _merge_state(state, out)
        return outputs

graph = GraphRunner()

In [15]:
# invoke the graph with a personalized initial state
graph.invoke({"graph_state" : "Hi, this is Lance.", "user_name": "Lance", "previous_graph_state": ""})


Starting graph run for Lance
---Node 1 (for Lance)---
Chosen next node: node_2
---Node 2 (happy for Lance)---
Chosen next node: node_3
---Node 3 (sad for Lance)---


/var/folders/wt/cx3jfjrx09gbzv39_r264my80000gn/T/ipykernel_8915/3002866323.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return {"graph_state": new, "diff": diff, "last_updated": datetime.utcnow().isoformat(), "mood": "neutral", "previous_graph_state": prev}
/var/folders/wt/cx3jfjrx09gbzv39_r264my80000gn/T/ipykernel_8915/3002866323.py:23: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return {"graph_state": new, "diff": diff, "last_updated": datetime.utcnow().isoformat(), "mood": "happy", "previous_graph_state": prev}
/var/folders/wt/cx3jfjrx09gbzv39_r264my80000gn/T/ipykernel_8915/3002866323.py:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and sched

[{'graph_state': 'Hi, this is Lance. I am',
  'diff': '--- \n+++ \n@@ -1 +1 @@\n-Hi, this is Lance.\n+Hi, this is Lance. I am',
  'last_updated': '2025-10-16T09:06:28.977271',
  'mood': 'neutral',
  'previous_graph_state': 'Hi, this is Lance.'},
 {'graph_state': 'Hi, this is Lance. I am happy!',
  'diff': '--- \n+++ \n@@ -1 +1 @@\n-Hi, this is Lance. I am\n+Hi, this is Lance. I am happy!',
  'last_updated': '2025-10-16T09:06:28.977320',
  'mood': 'happy',
  'previous_graph_state': 'Hi, this is Lance. I am'},
 {'graph_state': 'Hi, this is Lance. I am happy! sad!',
  'diff': '--- \n+++ \n@@ -1 +1 @@\n-Hi, this is Lance. I am happy!\n+Hi, this is Lance. I am happy! sad!',
  'last_updated': '2025-10-16T09:06:28.977348',
  'mood': 'sad',
  'previous_graph_state': 'Hi, this is Lance. I am happy!'}]